In [2]:
import sys
sys.path.append('/Users/hph/current_fits/cutout_figures/')
path = '/Users/hph/current_fits/cutout_figures/'
import numpy as np
import matplotlib.pyplot as plt
import aplpy
from astropy.io import fits
from astropy import wcs
import warnings
#import regions
import pyregion
from astropy.utils.console import ProgressBar

In [ ]:
def cutout(inputfile, xcenter, ycenter, width, height, cdelt = '', ranges = False, outfile = ''):

    ### This function creates a new fits file based on rectangular region information.
    ### xcenter, ycenter are the centers of the region in galactic longitude and latitude in degrees
    ### width and height are similarly longitude and latitude measurements in degrees
    
    path = '/Users/hph/cmzoom_catalog/catalog_acc/cutouts/'
    w = wcs.WCS(path+inputfile).celestial
    data, header = fits.getdata(path+inputfile, header = True)
        
    #if (header['CRVAL1'] != 0) or (header['CRVAL2'] != 0):
    #    raise ValueError('\nThe input CRVAL is not 0!\n')
    

    centre_x, centre_y = w.wcs_world2pix(xcenter, ycenter, 1)
    #print centre_x
    #print centre_y

    centre_x = int(centre_x)
    centre_y = int(centre_y)
    
    
    if cdelt == '':
        cdelt = np.mean( [np.absolute(header['CDELT1']), np.absolute(header['CDELT2'])] )
    cdelt = cdelt
        
        
    size_pixels_x = int(np.rint((width / 2.) / cdelt))
    size_pixels_y = int(np.rint((height / 2.) / cdelt))

    
    delta_y = np.array([centre_x - size_pixels_x, centre_x + size_pixels_x], dtype = int)
    delta_x = np.array([centre_y - size_pixels_y, centre_y + size_pixels_y], dtype = int)
    
    centre_x_new, centre_y_new  = w.wcs_pix2world( delta_y[0], delta_x[0], 1)
        
    data_cutout = data[ delta_x[0]: delta_x[1], delta_y[0]: delta_y[1] ]

    centre_x = ((delta_x[1] - delta_x[0]) / 2.) + delta_x[0]
    centre_y = ((delta_y[1] - delta_y[0]) / 2.) + delta_y[0]

    # centre_x_new, centre_y_new  = ((delta_x[1] - delta_x[0]) / 2.), ((delta_y[1] - delta_y[0]) / 2.)
    # centre_x_new_world, centre_y_new_world, velo  = w.wcs_pix2world(centre_y, centre_x, 0, 1)

    centre_x_new, centre_y_new = (header['CRPIX1'] - delta_y[0]), (header['CRPIX2'] - delta_x[0])

    if ranges: 
    
        stats = [np.nanmin(data_cutout), np.nanmax(data_cutout)]
        return stats
    
    if outfile != '':
        
        del header[29:72]
        del header['PC*']
        
        header['CRPIX1'] = size_pixels_x #float(centre_y) #_new
        header['CRPIX2'] = size_pixels_y #float(centre_x) #_new 
        header['CRVAL1'] = xcenter #float(centre_y_new_world) 
        header['CRVAL2'] = ycenter #float(centre_x_new_world)
    
        fits.writeto(path+outfile, data = data_cutout, header = header, overwrite = True)
        


def make_single(inputfits, outputfile, xcenter, ycenter, label, vmin=-1.14125e08, vmax =2.07402e+08, gamma=1.8):
    
    path = '/Users/hph/cmzoom_catalog/catalog_acc/cutouts/'
    
    fg_color='white'
    bg_color='black'
    fig = plt.figure(figsize=(5, 5))#,facecolor=bg_color, edgecolor=fg_color)
    subplot = aplpy.FITSFigure(inputfits, figure = fig, convention='calabretta')
    subplot.show_colorscale(vmin=vmin, vmax=vmax, stretch='power',exponent=gamma,cmap='inferno')
    subplot.set_nan_color(bg_color)
    subplot.ticks.set_xspacing(0.02)
    subplot.ticks.set_yspacing(0.02)
    subplot.ticks.set_color('black')
    subplot.tick_labels.set_xformat('d.dd')
    subplot.tick_labels.set_yformat('d.dd')
    subplot.add_label(0.35, 0.95, str(label), relative = True, weight = 'bold', size = 10, color = 'black')

    subplot.show_contour(path+'dendrogram_mask_pruned_rms1e7_dv3_dd1_dp50_pp6_pm2_gal_5-28-19.fits',
                         colors = 'white', levels = [0.0,1.0], linewidths = 0.7,
                         convention = 'calabretta', zorder = 10, linestyle = 'solid')
    subplot.axis_labels.set_xtext('Galactic Longitude (degrees)')
    subplot.axis_labels.set_ytext('Galactic Latitude (degrees)')
    subplot.ticks.show()
    subplot.add_scalebar(length=24./3600.)
    subplot.scalebar.set_label('1 pc')
    subplot.scalebar.set_color('white')
    #fig.tight_layout()
    fig.savefig(outputfile)
    plt.show()
    #plt.close('all')
    


In [ ]:
###################################################
### single use cutout:
###################################################
cutout('Continuum_mosaic_current_no_sgra_no_IHMSFR.fits', 
       0.3781046312, 0.04874423644, 
       0.1336001, 0.0728000, 
       outfile = 'dustridge_BCD.fits')


In [ ]:
###################################################
### single use postage figure:
###################################################
make_single(inputfits = path+'dustridge_BCD.fits',outputfile=path+'dustridge_BCD.pdf', 
            xcenter = 0.3781046312, ycenter = 0.04874423644, 
            label = 'Dust Ridge cloud B, C, and D')

In [ ]:
###################################################
### Load list of pyregions to cutout
###################################################
reglist = pyregion.open(path+'postage.reg')

In [ ]:
###################################################
### Generate cutout fits files for each region
###################################################
for reg_i in range(len(reglist)):
    reg = reglist[reg_i]
    cutout(inputfile='CMZoom_continuum_mosaic_without_sgra_pb_corrected_new.fits',
           xcenter = reg.coord_list[0], ycenter = reg.coord_list[1],
           width = reg.coord_list[2], height = reg.coord_list[3],
           outfile = "cutout_"+str(reg_i)+".fits")

In [ ]:
###################################################
### Generate postazge figure for each region
###################################################
for reg_i in range(len(reglist)):
    make_single(inputfits = path+'cutout_'+str(reg_i)+'.fits',outputfile=path+'cutout_'+str(reg_i)+'.pdf', 
                    xcenter = reg.coord_list[0], ycenter = reg.coord_list[1], 
                    label = '',gamma=gamma_arr[reg_i],vmax =vmax_arr[reg_i])
    plt.close()